In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
# from imblearn.over_sampling import ADASYN

# # Data loading and preprocessing
# df = pd.read_csv("../../data/data/train-metadata.csv", low_memory=False)

# missing_percentages = (df.isnull().sum() / len(df)) * 100
# columns_to_keep = missing_percentages[missing_percentages < 50].index
# skin_cancer_df = df[columns_to_keep]

# features_to_drop = [
#     'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
#     # 'mel_thick_mm', 'mel_mitotic_index',
#     # 'tbp_lv_dnn_lesion_confidence', 'tbp_lv_nevi_confidence',
#     'isic_id', 'patient_id', 'lesion_id', 'attribution', 'copyright_license'
# ]

# skin_cancer_df = skin_cancer_df.drop(columns=[col for col in features_to_drop if col in skin_cancer_df.columns])
# skin_cancer_df = skin_cancer_df.fillna(skin_cancer_df.mode().iloc[0])

# categorical_columns = skin_cancer_df.select_dtypes(include=['object']).columns
# for col in categorical_columns:
#     le = LabelEncoder()
#     skin_cancer_df[col] = le.fit_transform(skin_cancer_df[col])

# X = skin_cancer_df.drop('target', axis=1)
# y = skin_cancer_df['target']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# print(len(X_test))
# print(len(y_train))
# print("\nTraining set distribution:")
# print(y_train.value_counts())
# print("\nTest set distribution:")
# print(y_test.value_counts())

# # Balance using ADASYN
# adasyn = ADASYN(random_state=3)
# X_train_balanced, y_train_balanced = adasyn.fit_resample(X_train, y_train)

# # Train Decision Tree
# dt_balanced = DecisionTreeClassifier(class_weight='balanced', random_state=3)

# dt_balanced.fit(X_train_balanced, y_train_balanced)
# dt_balanced_pred = dt_balanced.predict(X_test)
# dt_balanced_pred_proba = dt_balanced.predict_proba(X_test)

# accuracy = accuracy_score(y_test, dt_balanced_pred)
# print(f"\nAccuracy: {accuracy:.3f}")

# print("\nConfusion Matrix:")
# print(confusion_matrix(y_test, dt_balanced_pred))

# print("\nDetailed Classification Report:")
# print(classification_report(y_test, dt_balanced_pred))

# auc_roc = roc_auc_score(y_test, dt_balanced_pred_proba[:, 1])
# print(f"\nAUC-ROC Score: {auc_roc:.3f}")

# feature_importance = pd.DataFrame({
#     'feature': X_train.columns,
#     'importance': dt_balanced.feature_importances_
# })
# feature_importance = feature_importance.sort_values('importance', ascending=False)

80212
320847

Training set distribution:
target
0    320536
1       311
Name: count, dtype: int64

Test set distribution:
target
0    80130
1       82
Name: count, dtype: int64

Accuracy: 0.993

Confusion Matrix:
[[79632   498]
 [   74     8]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     80130
           1       0.02      0.10      0.03        82

    accuracy                           0.99     80212
   macro avg       0.51      0.55      0.51     80212
weighted avg       1.00      0.99      1.00     80212


AUC-ROC Score: 0.546


In [ ]:
#imports
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, make_scorer, f1_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt

#reading csv files from download

df = pd.read_csv("../../data/data/train-metadata.csv", low_memory=False)

missing_percentages = (df.isnull().sum() / len(df)) * 100
columns_to_keep = missing_percentages[missing_percentages < 50].index
skin_cancer_df = df[columns_to_keep]

features_to_drop = [
    'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5',
    'isic_id', 'patient_id', 'lesion_id', 'attribution', 'copyright_license'
]

skin_cancer_df = skin_cancer_df.drop(columns=[col for col in features_to_drop if col in skin_cancer_df.columns])
skin_cancer_df = skin_cancer_df.fillna(skin_cancer_df.mode().iloc[0])

# Label encode all categorical columns
categorical_columns = skin_cancer_df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    le = LabelEncoder()
    skin_cancer_df[col] = le.fit_transform(skin_cancer_df[col])

X = skin_cancer_df.drop('target', axis=1)
y = skin_cancer_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 4: Reset indices of X_train and y_train to prevent conflicts during SMOTE
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

sm = SMOTE(sampling_strategy=0.1, random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
skin_cancer_decisiontree = DecisionTreeClassifier(class_weight='balanced', max_depth=5, min_samples_split=10,min_samples_leaf=5, random_state=42)
from sklearn.model_selection import cross_val_score
# Perform 5-Fold Cross-Validation
scores = cross_val_score(skin_cancer_decisiontree, X, y, cv=5, scoring='accuracy')

# Print the results
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation:", scores.std())
skin_cancer_decisiontree.fit(X_train_sm, y_train_sm)
y_predict = skin_cancer_decisiontree.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)
importances = skin_cancer_decisiontree.feature_importances_
feature_names = X.columns
sorted(zip(importances, feature_names), reverse=True)
# Evaluate predictions
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_predict))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_predict))
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get the predicted probabilities for the positive class
y_prob = skin_cancer_decisiontree.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Calculate the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Calculate the AUC score
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=2)  # Random guess line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()
